# Feature Selection of LSTM 

# Import libraries

In [2]:
!pip install pandas
!pip install sklearn
!pip install matplotlib

import matplotlib.pyplot as plt
import pandas as pd;
import numpy as np;
import sklearn
import time

# empty list to append metric values
mae_gen = []
mae_nor = []
mae_spi = []
rmse_gen = []
rmse_nor = []
rmse_spi = []
time_count = []
y_pred_list = []

# Import keras libraries, packages and data:

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras import initializers
from keras import optimizers
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# parameters
steps = 336
n_hidden = 2
units = 100
batch_size = 96
epochs = 100

# months to evaluate model on
date = 2018050000

# for later use
#features_num = 14

# import data
data_full = pd.read_csv('Data_set_1_smaller_(1).csv', index_col = 0)

# set predictive window according with tuning best results
data_full = data_full.loc[data_full.index > date, :]

# reset index
data_full.reset_index(inplace = True)
data_full.drop('index', axis = 1, inplace = True)

# fill nan values in the whole data set
data_full.fillna(data_full.mean(), inplace = True)

# Functions for LSTM:

In [4]:
# function to split data into correct shape for RNN
def split_data(X, y, steps):
    X_, y_ = list(), list()
    for i in range(steps, len(y)):
        X_.append(X[i - steps : i, :])
        y_.append(y[i]) 
    return np.array(X_), np.array(y_)

# function to cut data set so it can be divisible by the batch_size
def cut_data(data, batch_size):
     # see if it is divisivel
    condition = data.shape[0] % batch_size
    if condition == 0:
        return data
    else:
        return data[: -condition]
   

# FOR SPIKES: function to split data into correct shape for RNN 
def split_data_spike(shade_test, steps):
    y_spike_occ = list()
    upper_lim = list()
    lower_lim = list()
    for i in range(steps, len(shade_test.index)):
        y_spike_occ.append(shade_test['spike_occurance'][i])
        upper_lim.append(shade_test['spike_upperlim'][i])
        lower_lim.append(shade_test['spike_lowerlim'][i])
    return np.array(y_spike_occ), np.array(upper_lim), np.array(lower_lim)

# FOR SPIKES: function to cut data set so it can be divisible by the batch_size
def cut_data_spike(data, batch_size):
     # see if it is divisivel
    condition = data.shape[0] % batch_size
    if condition == 0:
        return data
    else:
        return data[: -condition]

    
# function defining regressor
def regressor_tunning(kernel_initializer = 'he_uniform',
                      bias_initializer = initializers.Ones()):
    model = Sequential()
    if n_hidden == 1:
        model.add(LSTM(units = units,                    
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
    else:
        model.add(LSTM(units = units,                    
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       return_sequences = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
        model.add(LSTM(units = units, 
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))
    optimizer = optimizers.RMSprop()
    model.compile(loss = 'mse', metrics = ['mse', 'mae'], optimizer = optimizer)
    return model

# First prediction with one feature:

In [5]:
start_time = time.time()

# get data ready with one feature
data = data_full.loc[:,['PrevDay', 'Offers']]

# divide data into train and test 
data_train, data_test = train_test_split(
             data, test_size = 0.15, shuffle=False) 

sc_X = MinMaxScaler()
data_train = sc_X.fit_transform(data_train)
data_test = sc_X.transform(data_test)

# divide features and labels
X_train = data_train[:, 0] 
y_train = data_train[:, -1]
X_test = data_test[:, 0] 
y_test = data_test[:, -1]

# divide data into train and test 
X_train, X_val, y_train, y_val = train_test_split(
             X_train, y_train, test_size = 0.15, shuffle=False)

# function to split data into correct shape for RNN for 1 feature only
def split_data_1(X, y, steps):
    X_, y_ = list(), list()
    for i in range(steps, len(y)):
        X_.append(X[i - steps : i])
        y_.append(y[i]) 
    return np.array(X_), np.array(y_)

# put data into correct shape
X_train, y_train = split_data_1(X_train, y_train, steps)
X_test, y_test = split_data_1(X_test, y_test, steps)
X_val, y_val = split_data_1(X_val, y_val, steps)

X_train = cut_data(X_train, batch_size)
y_train = cut_data(y_train, batch_size)
X_test = cut_data(X_test, batch_size)
y_test = cut_data(y_test, batch_size)
X_val = cut_data(X_val, batch_size)
y_val = cut_data(y_val, batch_size)

# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

features_num = 1

model = regressor_tunning()

# fitting the LSTM to the training set
history = model.fit(X_train,
                    y_train, 
                    batch_size = batch_size, 
                    epochs = epochs,
                    shuffle = False, 
                    validation_data = (X_val, y_val))

# make new predicitons with test set
y_pred = model.predict(X_test, batch_size = batch_size)
    
# prices col = 15 (inverso should not be used as scalling was made with the whole data set)
y_pred = (y_pred * sc_X.data_range_[-1]) + (sc_X.data_min_[-1])
y_test = (y_test * sc_X.data_range_[-1]) + (sc_X.data_min_[-1])

# save prediciton
y_pred_list.append(y_pred)

# smal adjustment
y_test = pd.Series(y_test)
y_test.replace(0, 0.0001,inplace = True)
 
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

rmse_error = mse(y_test, y_pred, squared = False)
mae_error = mae(y_test, y_pred)
  
rmse_gen.append(rmse_error)
mae_gen.append(mae_error)
    
# =============================================================================
# Metrics evaluation on spike and normal regions
# =============================================================================

# (Need to process data with spike occurences the same way as features)

# download data for shaded area
data = pd.read_csv('Spike_binary_1std.csv', index_col = 0)

# set predictive window according with tuning best results
data = data.loc[data.index > date, :]

# make sure shaded area will correspond to values outputed by LSTM
data.reset_index(drop = True, inplace = True)

# fill_nan is already made - so lets split data into test and train
from sklearn.model_selection import train_test_split

# divide data into train and test 
shade_train, shade_test = train_test_split(
         data, test_size = 0.15, shuffle = False)

# reset index of testing data
shade_test.reset_index(drop = True, inplace = True)

y_spike_occ, spike_upperlim, spike_lowerlim = split_data_spike(shade_test, steps)
y_spike_occ = cut_data_spike(y_spike_occ, batch_size)
spike_upperlim = cut_data_spike(spike_upperlim, batch_size)
spike_lowerlim = cut_data_spike(spike_lowerlim, batch_size)

# =============================================================================
# METRICS EVALUATION (2) on spike regions
# =============================================================================

# select y_pred and y_test only for regions with spikes
y_test_spike = (y_test.T * y_spike_occ).T
y_pred_spike = (y_pred.T * y_spike_occ).T
y_test_spike = y_test_spike[y_test_spike != 0]
y_pred_spike = y_pred_spike[y_pred_spike != 0]
    
# calculate metric
rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
mae_spike = mae(y_test_spike, y_pred_spike)
    
rmse_spi.append(rmse_spike)
mae_spi.append(mae_spike)
    
# =============================================================================
# METRIC EVALUATION (3) on normal regions
# =============================================================================

# inverse y_spike_occ so the only normal occurences are chosen
y_normal_occ = (y_spike_occ - 1) * (-1)
 
# sanity check
y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 
  
# select y_pred and y_test only for normal regions
y_test_normal = (y_test.T * y_normal_occ).T
y_pred_normal = (y_pred.T * y_normal_occ).T
y_test_normal = y_test_normal[y_test_normal != 0.00]
y_pred_normal = y_pred_normal[y_pred_normal != 0.00]
    
# calculate metric
rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
mae_normal = mae(y_test_normal, y_pred_normal)
    
rmse_nor.append(rmse_normal)
mae_nor.append(mae_normal)
    
elapsed_time = time.time() - start_time

time_count.append(elapsed_time)

Epoch 1/100
85/85 [==============================] - 203s 2s/step - loss: 0.1703 - mse: 0.1703 - mae: 0.2998 - val_loss: 0.0525 - val_mse: 0.0525 - val_mae: 0.2267
Epoch 2/100
85/85 [==============================] - 5573s 66s/step - loss: 0.0287 - mse: 0.0287 - mae: 0.1327 - val_loss: 0.0032 - val_mse: 0.0032 - val_mae: 0.0465
Epoch 3/100
85/85 [==============================] - 205s 2s/step - loss: 0.0063 - mse: 0.0063 - mae: 0.0617 - val_loss: 0.0036 - val_mse: 0.0036 - val_mae: 0.0503
Epoch 4/100
85/85 [==============================] - 199s 2s/step - loss: 0.0022 - mse: 0.0022 - mae: 0.0352 - val_loss: 0.0023 - val_mse: 0.0023 - val_mae: 0.0345
Epoch 5/100
83/85 [============================>.] - ETA: 4s - loss: 0.0012 - mse: 0.0012 - mae: 0.0244

KeyboardInterrupt: 

# Loop to continue testing other features:

In [ ]:
# features list; order made according to Linear Regression FS
features_list = ['APXP', 
                 'LOLP',  
                 'In_gen',
                 'Ren_R',
                 'DA_imb_France', 
                 'Rene',
                 'ratio_offers_vol',
                 'DA_price_france',
                 'TSDF',
                 'dino_bin',
                 'DA_margin',
                 'Im_Pr']

best_score = rmse_error

# LOOP STARTS
for i in features_list:
    start_time = time.time()
    
    # data recovery for in case there is no improvement
    data_recovery = data
    
    # update feature number
    features_num = features_num + 1
    
    # add new feature
    data = pd.concat([data, data_full.loc[:,i]], axis = 1)    

    # divide data into train and test 
    data_train, data_test = train_test_split(
             data, test_size = 0.15, shuffle=False)    
    
    from sklearn.preprocessing import MinMaxScaler

    # data scaling  (including offer (y))
    sc_X = MinMaxScaler()
    data_train = sc_X.fit_transform(data_train)
    data_test = sc_X.transform(data_test)
    
    # divide features and labels
    X_train = data_train[:, 0: features_num] 
    y_train = data_train[:, -1]
    X_test = data_test[:, 0:features_num] 
    y_test = data_test[:, -1] 

    # divide data into train and test 
    X_train, X_val, y_train, y_val = train_test_split(
             X_train, y_train, test_size = 0.15, shuffle=False)

    # put data into correct shape
    X_train, y_train = split_data(X_train, y_train, steps)
    X_test, y_test = split_data(X_test, y_test, steps)
    X_val, y_val = split_data(X_val, y_val, steps)

    X_train = cut_data(X_train, batch_size)
    y_train = cut_data(y_train, batch_size)
    X_test = cut_data(X_test, batch_size)
    y_test = cut_data(y_test, batch_size)
    X_val = cut_data(X_val, batch_size)
    y_val = cut_data(y_val, batch_size)

    model = regressor_tunning()
    
    # fitting the LSTM to the training set
    history = model.fit(X_train,
                        y_train, 
                        batch_size = batch_size, 
                        epochs = epochs,
                        shuffle = False, 
                        validation_data = (X_val, y_val))
    
    model.reset_states()
    
    # make new predicitons with test set
    y_pred = model.predict(X_test, batch_size = batch_size)
    
    # prices col = 15 (inverso should not be used as scalling was made with the whole data set)
    y_pred = (y_pred * sc_X.data_range_[-1]) + (sc_X.data_min_[-1])
    y_test = (y_test * sc_X.data_range_[-1]) + (sc_X.data_min_[-1])

    # save prediciton
    y_pred_list.append(y_pred)
    
    # smal adjustment
    y_test = pd.Series(y_test)
    y_test.replace(0, 0.0001,inplace = True)

    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae

    rmse_error = mse(y_test, y_pred, squared = False)
    mae_error = mae(y_test, y_pred)

    rmse_gen.append(rmse_error)
    mae_gen.append(mae_error)

    # =============================================================================
    # Metrics evaluation on spike and normal regions
    # =============================================================================

    # (Need to process data with spike occurences the same way as features)

    # download data for shaded area
    data = pd.read_csv('Spike_binary_1std.csv', index_col = 0)

    # set predictive window according with tuning best results
    data = data.loc[data.index > date, :]

    # make sure shaded area will correspond to values outputed by LSTM
    data.reset_index(drop = True, inplace = True)

    # fill_nan is already made - so lets split data into test and train
    from sklearn.model_selection import train_test_split

    # divide data into train and test 
    shade_train, shade_test = train_test_split(
             data, test_size = 0.15, shuffle = False)

    # reset index of testing data
    shade_test.reset_index(drop = True, inplace = True)

    y_spike_occ, spike_upperlim, spike_lowerlim = split_data_spike(shade_test, steps)
    y_spike_occ = cut_data_spike(y_spike_occ, batch_size)
    spike_upperlim = cut_data_spike(spike_upperlim, batch_size)
    spike_lowerlim = cut_data_spike(spike_lowerlim, batch_size)

    # =============================================================================
    # METRICS EVALUATION (2) on spike regions
    # =============================================================================

    # select y_pred and y_test only for regions with spikes
    y_test_spike = (y_test.T * y_spike_occ).T
    y_pred_spike = (y_pred.T * y_spike_occ).T
    y_test_spike = y_test_spike[y_test_spike != 0]
    y_pred_spike = y_pred_spike[y_pred_spike != 0]

    # calculate metric
    rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
    mae_spike = mae(y_test_spike, y_pred_spike)

    rmse_spi.append(rmse_spike)
    mae_spi.append(mae_spike)

    # =============================================================================
    # METRIC EVALUATION (3) on normal regions
    # =============================================================================

    # inverse y_spike_occ so the only normal occurences are chosen
    y_normal_occ = (y_spike_occ - 1) * (-1)

    # sanity check
    y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 

    # select y_pred and y_test only for normal regions
    y_test_normal = (y_test.T * y_normal_occ).T
    y_pred_normal = (y_pred.T * y_normal_occ).T
    y_test_normal = y_test_normal[y_test_normal != 0.00]
    y_pred_normal = y_pred_normal[y_pred_normal != 0.00]

    # calculate metric
    rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
    mae_normal = mae(y_test_normal, y_pred_normal)

    rmse_nor.append(rmse_normal)
    mae_nor.append(mae_normal)

    elapsed_time = time.time() - start_time

    time_count.append(elapsed_time)
    
    # condition of improvement for FS
    if best_score < rmse_gen[-1]:
        data = data_recovery
        features_num = features_num - 1
    else:
        data = data
        best_score = rmse_error

In [ ]:
results = pd.DataFrame({                        
                        'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor,
    
                        'time': time_count})

In [ ]:
def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]

results.style.apply(highlight_min)

In [ ]:
results.min()

In [ ]:
features_list = ['PrevDay',
                 'APXP', 
                 'LOLP',  
                 'In_gen',
                 'Ren_R',
                 'DA_imb_France', 
                 'Rene',
                 'ratio_offers_vol',
                 'DA_price_france',
                 'TSDF',
                 'dino_bin',
                 'DA_margin',
                 'Im_Pr']

results = pd.DataFrame({'Feature added': features_list,
                        
                        'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor,
    
                        'time': time_count})